In [ ]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-05_UTC_with_lookback.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)


class EmaCrossover(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
        ('position_pct', 0.95),
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

        # Simple counters
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        self.order= None  # To keep track of pending orders

    def next(self):
        if len(self) < self.params.long_period:
            return
        
        # Skip if we have pending orders
        if self.order:
            return
        
        # Get current position
        position = self.position.size

        if self.crossover < 0:
            print(f"🔥BEARISH CROSSOVER - Going SHORT")
             # Calculate how many units you can afford
            cash = self.broker.getcash()
            price = self.data.close[0]
            size = (cash * self.params.position_pct) / price
            
            # print(f"Cash: ${cash:.2f}, Price: ${price:.2f}, Size: {size}")
            self.order = self.sell(size=size)
        
        elif self.crossover > 0 and position < 0:
            print(f"🔥 BULLISH CROSSOVER - Closing SHORT")

            # Close the entire short position
            # print(
                # f"Current short position: {position} ---- Price: ${self.data.close[0]:.2f}")
            self.order = self.buy(size=abs(position))



    def stop(self):
        """Called at the end of strategy - force close any open positions"""
        position = self.position.size

        if position != 0:
            print(f"🔚 END OF DATA - Force closing position: {position} units")
            if position > 0:
                # Close LONG position
                self.sell(size=abs(position))
            else:
                # Close SHORT position
                self.buy(size=abs(position))
            print("=" * 50)
    
    def notify_order(self, order):
        if order.status == order.Completed:
            if order.isbuy():
                print(f"BUY EXECUTED at ${order.executed.price:.2f}")
            elif order.issell():
                print(f"SELL EXECUTED at ${order.executed.price:.2f}")
        elif order.status == order.Margin:
            print(f"   ❌ ORDER REJECTED - Insufficient Margin/Cash")

        elif order.status == order.Rejected:
            print(f"   ❌ ORDER REJECTED - {order.info}")

        elif order.status == order.Canceled:
            print(f"   ⚠️ ORDER CANCELED")
            
        """Reset self.order when order completes"""
        if order.status in [order.Completed, order.Canceled, order.Rejected , order.Margin]:
            self.order = None  # ✅ This allows new orders to be created

    def notify_trade(self, trade):
        """Handle trade notifications"""
        if trade.isclosed:
            self.total_trades += 1
            pnl = trade.pnl
            self.total_pnl += pnl

            # Determine trade direction
            direction = "LONG" if trade.size > 0 else "SHORT"

            if pnl > 0:
                self.winning_trades += 1
                print(f"   ✅ WINNING {direction} TRADE - P&L: ${pnl:.2f}")
            else:
                print(f"   ❌ LOSING {direction} TRADE - P&L: ${pnl:.2f}")

            # Print running statistics
            win_rate = (self.winning_trades / self.total_trades) * 100
            print(
                f"   📊 Total: {self.total_trades} | Win Rate: {win_rate:.1f}% | Total P&L: ${self.total_pnl:.2f}")
            print("=" * 50)

  

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(EmaCrossover)


cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission


# print(f"Starting Value: ${cerebro.broker.getvalue():.2f}")
cerebro.run()

print("=" * 50)
print(f"🏁 Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")

🔥BEARISH CROSSOVER - Going SHORT
SELL EXECUTED at $104600.40
🔥 BULLISH CROSSOVER - Closing SHORT
____________________________________
BUY EXECUTED at $104854.90
   ❌ LOSING SHORT TRADE - P&L: $-23.11
   📊 Total: 1 | Win Rate: 0.0% | Total P&L: $-23.11
🔥BEARISH CROSSOVER - Going SHORT
SELL EXECUTED at $104525.00
🔥 BULLISH CROSSOVER - Closing SHORT
____________________________________
BUY EXECUTED at $104772.70
   ❌ LOSING SHORT TRADE - P&L: $-22.42
   📊 Total: 2 | Win Rate: 0.0% | Total P&L: $-45.53
🔥BEARISH CROSSOVER - Going SHORT
SELL EXECUTED at $104230.90
🔚 END OF DATA - Force closing position: -0.0903827682491413 units
🏁 Final Portfolio Value: $10176.06
